In [2]:
import mediapipe as mp
import cv2
import numpy as np

In [3]:
mp_drawing = mp.solutions.drawing_utils 
mp_pose = mp.solutions.pose

1.Make Detections

In [4]:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, image = cap.read()

        #Recolor feed
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        #Make Detections
        result = pose.process(image)

        #Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image, result.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        cv2.imshow('Raw webcam feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [21]:
cap = cv2.VideoCapture(0)

height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
fps = cap.get(cv2.CAP_PROP_FPS)
videoWriter = cv2.VideoWriter('press.avi', cv2.VideoWriter_fourcc('P','I','M','1'), fps, (int(width), int(height)))

while cap.isOpened():
    ret, frame = cap.read()

    try:
        cv2.imshow('Press', frame)
        videoWriter.write(frame)
    except Exception as e:
        break

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
videoWriter.release()
cv2.destroyAllWindows()

2.Capture Landmark and Export to CSV

In [4]:
import csv
import os
import numpy as np
from matplotlib import pyplot as plt

In [5]:
landmarks = ['class']
for val in range(1,33+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [6]:
landmarks[1:]

['x1',
 'y1',
 'z1',
 'v1',
 'x2',
 'y2',
 'z2',
 'v2',
 'x3',
 'y3',
 'z3',
 'v3',
 'x4',
 'y4',
 'z4',
 'v4',
 'x5',
 'y5',
 'z5',
 'v5',
 'x6',
 'y6',
 'z6',
 'v6',
 'x7',
 'y7',
 'z7',
 'v7',
 'x8',
 'y8',
 'z8',
 'v8',
 'x9',
 'y9',
 'z9',
 'v9',
 'x10',
 'y10',
 'z10',
 'v10',
 'x11',
 'y11',
 'z11',
 'v11',
 'x12',
 'y12',
 'z12',
 'v12',
 'x13',
 'y13',
 'z13',
 'v13',
 'x14',
 'y14',
 'z14',
 'v14',
 'x15',
 'y15',
 'z15',
 'v15',
 'x16',
 'y16',
 'z16',
 'v16',
 'x17',
 'y17',
 'z17',
 'v17',
 'x18',
 'y18',
 'z18',
 'v18',
 'x19',
 'y19',
 'z19',
 'v19',
 'x20',
 'y20',
 'z20',
 'v20',
 'x21',
 'y21',
 'z21',
 'v21',
 'x22',
 'y22',
 'z22',
 'v22',
 'x23',
 'y23',
 'z23',
 'v23',
 'x24',
 'y24',
 'z24',
 'v24',
 'x25',
 'y25',
 'z25',
 'v25',
 'x26',
 'y26',
 'z26',
 'v26',
 'x27',
 'y27',
 'z27',
 'v27',
 'x28',
 'y28',
 'z28',
 'v28',
 'x29',
 'y29',
 'z29',
 'v29',
 'x30',
 'y30',
 'z30',
 'v30',
 'x31',
 'y31',
 'z31',
 'v31',
 'x32',
 'y32',
 'z32',
 'v32',
 'x33',
 'y3

In [25]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [7]:
def export_landmark(result, action):
    try:
        keypoints = np.array([[res.x, res.y, res.z, res.visibility] for res in result.pose_landmarks.landmark]).flatten().tolist()
        keypoints.insert(0, action)

        with open('coords.csv',mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        pass

In [27]:
export_landmark(result, "up")

In [ ]:
cap = cv2.VideoCapture('press.avi')
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, image = cap.read()

        #Recolor feed
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        #Make Detections
        result = pose.process(image)

        #Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image, result.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        k = cv2.waitKey(1)
        if k == 117:
            export_landmark(result, 'up')
        if k == 100:
            export_landmark(result, 'down')


        cv2.imshow('Raw webcam feed', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

3.Train Custom Model Using Scikit Learn

3.1 Read in collected data and process

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [9]:
df = pd.read_csv('coords.csv')

In [10]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,up,0.572532,0.219501,-2.164514,0.998732,0.645755,0.106812,-2.197974,0.996956,0.677718,...,1.616902,0.045638,0.679216,1.629449,1.736612,0.040204,0.652867,1.659249,1.311589,0.041659
1,up,0.556637,0.126634,-0.078371,0.998010,0.550914,0.105974,-0.062308,0.996788,0.549429,...,-0.014706,0.939049,0.553890,0.886272,0.353643,0.862102,0.559419,0.940621,-0.119284,0.966660
2,up,0.555191,0.123107,-0.078079,0.998953,0.548255,0.102541,-0.061358,0.998509,0.546665,...,-0.038162,0.936512,0.554154,0.886587,0.333253,0.873246,0.560862,0.940187,-0.145768,0.967695
3,down,0.544072,0.114125,-0.145885,0.994389,0.536135,0.092615,-0.126509,0.987798,0.534789,...,-0.025520,0.938964,0.557736,0.892450,0.305521,0.781440,0.560785,0.940381,-0.156524,0.961771
4,down,0.544017,0.114094,-0.145757,0.994277,0.536121,0.092539,-0.126375,0.987413,0.534782,...,-0.025136,0.940168,0.557986,0.892522,0.311964,0.781350,0.560872,0.940453,-0.156719,0.962502


In [11]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
315,down,0.519302,0.010166,-0.134836,0.998369,0.528045,-0.017540,-0.170251,0.998622,0.530554,...,0.497208,0.061440,0.528666,1.244810,0.017372,0.101211,0.528562,1.171972,0.422061,0.020929
316,up,0.504486,0.027654,-0.005380,0.998546,0.511085,-0.003689,-0.048618,0.998991,0.514217,...,0.366031,0.051487,0.554490,1.199282,-0.057411,0.087479,0.535852,1.143240,0.315476,0.019300
317,up,0.504965,0.030119,-0.016276,0.998752,0.511752,0.002275,-0.063868,0.999164,0.514957,...,0.374597,0.075913,0.552054,1.189770,-0.010841,0.102956,0.546317,1.127380,0.327724,0.026438
318,up,0.504977,0.030675,-0.017907,0.998722,0.511567,0.003618,-0.064486,0.999150,0.514482,...,0.356011,0.074359,0.551730,1.193364,-0.011235,0.101848,0.548059,1.135098,0.308430,0.025869
319,down,0.523624,-0.021167,-0.528726,0.999430,0.534441,-0.046153,-0.533279,0.999482,0.543627,...,0.717913,0.318335,0.600500,1.221621,0.210562,0.277454,0.541886,1.151816,0.582184,0.208415


In [12]:
df[df['class']=='up']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,up,0.572532,0.219501,-2.164514,0.998732,0.645755,0.106812,-2.197974,0.996956,0.677718,...,1.616902,0.045638,0.679216,1.629449,1.736612,0.040204,0.652867,1.659249,1.311589,0.041659
1,up,0.556637,0.126634,-0.078371,0.998010,0.550914,0.105974,-0.062308,0.996788,0.549429,...,-0.014706,0.939049,0.553890,0.886272,0.353643,0.862102,0.559419,0.940621,-0.119284,0.966660
2,up,0.555191,0.123107,-0.078079,0.998953,0.548255,0.102541,-0.061358,0.998509,0.546665,...,-0.038162,0.936512,0.554154,0.886587,0.333253,0.873246,0.560862,0.940187,-0.145768,0.967695
6,up,0.542514,0.112984,-0.083399,0.997618,0.536134,0.093447,-0.066297,0.995917,0.535368,...,-0.012353,0.938063,0.554178,0.890328,0.326668,0.854518,0.561984,0.939475,-0.122262,0.965443
7,up,0.541198,0.112923,-0.088066,0.998814,0.534699,0.093400,-0.072637,0.998173,0.533925,...,-0.002546,0.946483,0.554901,0.889242,0.345404,0.879516,0.562563,0.939621,-0.110935,0.971599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,up,0.508185,0.030871,-0.014367,0.998636,0.514413,0.006038,-0.060531,0.999138,0.516717,...,0.361481,0.051722,0.552534,1.196106,-0.043328,0.084734,0.541738,1.135318,0.316742,0.018786
311,up,0.507693,0.030838,-0.014001,0.998676,0.513917,0.005943,-0.059987,0.999169,0.516201,...,0.355316,0.051571,0.552301,1.196121,-0.040043,0.085869,0.542067,1.135837,0.310180,0.018855
316,up,0.504486,0.027654,-0.005380,0.998546,0.511085,-0.003689,-0.048618,0.998991,0.514217,...,0.366031,0.051487,0.554490,1.199282,-0.057411,0.087479,0.535852,1.143240,0.315476,0.019300
317,up,0.504965,0.030119,-0.016276,0.998752,0.511752,0.002275,-0.063868,0.999164,0.514957,...,0.374597,0.075913,0.552054,1.189770,-0.010841,0.102956,0.546317,1.127380,0.327724,0.026438


In [13]:
X = df.drop('class', axis=1) # features
Y = df['class'] # target value

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=888)

In [15]:
Y_test

114    down
212      up
174      up
213      up
119    down
       ... 
33       up
133      up
184    down
57       up
145      up
Name: class, Length: 64, dtype: object

3.2 Train Machine Learning Classification Model

In [16]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [17]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [18]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, Y_train)
    fit_models[algo] = model

c:\Program Files\Python311\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Program Files\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Program Files\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Program Files\Python311\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` inst

In [19]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [20]:
fit_models['rc'].predict(X_test)

c:\Program Files\Python311\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Program Files\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Program Files\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


array(['down', 'up', 'up', 'up', 'down', 'down', 'up', 'down', 'up', 'up',
       'down', 'up', 'down', 'down', 'down', 'up', 'up', 'up', 'down',
       'down', 'up', 'up', 'down', 'up', 'up', 'down', 'up', 'down', 'up',
       'down', 'up', 'up', 'up', 'up', 'down', 'up', 'up', 'down', 'down',
       'up', 'up', 'up', 'up', 'up', 'up', 'down', 'up', 'up', 'up', 'up',
       'down', 'up', 'up', 'down', 'up', 'down', 'up', 'up', 'up', 'up',
       'up', 'down', 'up', 'up'], dtype='<U4')

3.3 Evalutate and Serialize Model

In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score # Accuracy metrics
import pickle

In [22]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(Y_test.values, yhat),
          precision_score(Y_test.values, yhat, average="binary", pos_label="up"),
          recall_score(Y_test.values, yhat, average="binary", pos_label="up"))

lr 1.0 1.0 1.0
rc 1.0 1.0 1.0
rf 1.0 1.0 1.0
gb 1.0 1.0 1.0


c:\Program Files\Python311\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Program Files\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Program Files\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Program Files\Python311\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` inst

In [23]:
yhat = fit_models['rf'].predict(X_test)

c:\Program Files\Python311\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Program Files\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Program Files\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [24]:
yhat[:10]

array(['down', 'up', 'up', 'up', 'down', 'down', 'up', 'down', 'up', 'up'],
      dtype=object)

In [25]:
with open('barbell_curl.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

4.Make Detections with Model

START FROM HERE

In [1]:
import mediapipe as mp
import cv2
import numpy as np
import csv
import os
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score # Accuracy metrics
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
mp_drawing = mp.solutions.drawing_utils 
mp_pose = mp.solutions.pose

In [3]:
landmarks = ['class']
for val in range(1,33+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [4]:
with open('barbell_curl.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
cap = cv2.VideoCapture(0)
counter = 0
current_stage = ''

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

    while cap.isOpened():
        ret, frame = cap.read()

        #Recolor feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        #make Detections
        result = pose.process(image)

        #Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image, result.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
        
        try: 
            row = np.array([[res.x, res.y, res.z, res.visibility] for res in result.pose_landmarks.landmark]).flatten().tolist()
            X = pd.DataFrame([row], columns = landmarks[1:]) 
            body_language_prob = model.predict_proba(X)[0]
            body_language_class = model.predict(X)[0]
            print(body_language_class, body_language_prob) 

            if body_language_class =="down" and body_language_prob[body_language_prob.argmax()] >= 0.7: 
                current_stage = "down" 
            elif current_stage == "down" and body_language_class == "up" and body_language_prob[body_language_prob.argmax()] >= 0.7:
                current_stage = "up" 
                counter += 1
                print(current_stage) 
            

            # Get status box
            cv2.rectangle(image, (0,0), (250,60), (245,117,16), -1)

            # Display Class
            cv2.putText(image, 'CLASS', (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0], (90,40), cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

            # Display Probability
            cv2.putText(image, 'PROB', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image,str(round(body_language_prob[np.argmax(body_language_prob)],2)), (10,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

            # Display Counter
            cv2.putText(image, 'COUNT', (180,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), (175,40), cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

        
        except Exception as e:
            pass

        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break


cap.release()
cv2.destroyAllWindows()